## VGG TRANSFER LEARNING 

### PREPARE DATASET 





In [1]:
# Data prepare: 2 classes [5,21] for quick check 
# VGG data prepare

import pandas as pd
import numpy as np
import csv
import json
import random
import matplotlib.pyplot as plt
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


train_file = '/root/data/S10.json'
with open(train_file) as data_file:
    train_anns=json.load(data_file)


img_5=[]
for i in train_anns['annotations']:
    if i['category_id'] == 5:
        img_5.append(i['image_id']+'.JPG')
img_5_1w = random.sample(img_5, 1300)


img_21=[]
for i in train_anns['annotations']:
    if i['category_id'] == 21:
        img_21.append(i['image_id']+'.JPG')
img_21_1w = random.sample(img_21,1300)

img_list = img_5_1w+img_21_1w

label_dict = {}
for i in train_anns['annotations']:
    i_new = i['image_id'] + '.JPG'
    if i_new in img_list:
        label_dict[i_new]=i['category_id']
        
df = pd.DataFrame.from_dict(label_dict,orient='index',columns=['label'])
df = df.reset_index().rename(columns={'index':'image_id'})
print(df.head())

df['label'].value_counts().plot.bar()
plt.show()

df.label.unique()

df.loc[df.label == 39, 'label']=21

df.label.unique()

df['label'].value_counts().plot.bar()
plt.show()

Using TensorFlow backend.


                                 image_id  label
0  S10/B03/B03_R1/S10_B03_R1_IMAG0958.JPG      5
1  S10/B03/B03_R1/S10_B03_R1_IMAG0992.JPG      5
2  S10/B03/B03_R1/S10_B03_R1_IMAG1049.JPG      5
3  S10/B03/B03_R1/S10_B03_R1_IMAG1115.JPG      5
4  S10/B03/B03_R1/S10_B03_R1_IMAG1307.JPG      5


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [4]:
df.label.unique()

array([ 5, 21])

In [5]:
df.shape

(2600, 2)

In [6]:
type(df)

pandas.core.frame.DataFrame

In [7]:
df.loc[df.label == 5, 'label']=0
df.loc[df.label == 21, 'label']=1

In [8]:
df

,image_id,label
0,S10/B03/B03_R1/S10_B03_R1_IMAG0958.JPG,0
1,S10/B03/B03_R1/S10_B03_R1_IMAG0992.JPG,0
2,S10/B03/B03_R1/S10_B03_R1_IMAG1049.JPG,0
3,S10/B03/B03_R1/S10_B03_R1_IMAG1115.JPG,0
4,S10/B03/B03_R1/S10_B03_R1_IMAG1307.JPG,0
...,...,...
2595,S10/T11/T11_R2/S10_T11_R2_IMAG1007.JPG,0
2596,S10/T11/T11_R2/S10_T11_R2_IMAG1020.JPG,0
2597,S10/T11/T11_R2/S10_T11_R2_IMAG1084.JPG,0
2598,S10/T11/T11_R2/S10_T11_R2_IMAG1098.JPG,0


In [9]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=523)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [10]:
train_df[:2]

,image_id,label
0,S10/O09/O09_R2/S10_O09_R2_IMAG1908.JPG,0
1,S10/C08/C08_R1/S10_C08_R1_IMAG0262.JPG,1


In [11]:
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import random

check the not existed images

In [12]:
imglist = df['image_id'].tolist()
path = '/root/data/'

count = 0
nonelist = []
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path)
        count+=1
    except:
        nonelist.append(img_name)   

In [13]:
len(nonelist)

18

In [14]:
nonelist

['S10/E06/E06_R1/S10_E06_R1_IMAG0874.JPG',
 'S10/E06/E06_R1/S10_E06_R1_IMAG0875.JPG',
 'S10/E06/E06_R1/S10_E06_R1_IMAG0876.JPG',
 'S10/I06/I06_R1/S10_I06_R1_IMAG1192.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0489.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0490.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0491.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0492.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0493.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0494.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0495.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0496.JPG',
 'S10/K09/K09_R1/S10_K09_R1_IMAG0497.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0261.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0262.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0263.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0268.JPG',
 'S10/M08/M08_R1/S10_M08_R1_IMAG0269.JPG']

In [21]:
nonelist1 = []
for i in nonelist:
    j = i[:]
    nonelist1.append(j)

In [22]:
nonelist1

['S10_E06_R1_IMAG0874.JPG',
 'S10_E06_R1_IMAG0875.JPG',
 'S10_E06_R1_IMAG0876.JPG',
 'S10_I06_R1_IMAG1192.JPG',
 'S10_K09_R1_IMAG0489.JPG',
 'S10_K09_R1_IMAG0490.JPG',
 'S10_K09_R1_IMAG0491.JPG',
 'S10_K09_R1_IMAG0492.JPG',
 'S10_K09_R1_IMAG0493.JPG',
 'S10_K09_R1_IMAG0494.JPG',
 'S10_K09_R1_IMAG0495.JPG',
 'S10_K09_R1_IMAG0496.JPG',
 'S10_K09_R1_IMAG0497.JPG',
 'S10_M08_R1_IMAG0261.JPG',
 'S10_M08_R1_IMAG0262.JPG',
 'S10_M08_R1_IMAG0263.JPG',
 'S10_M08_R1_IMAG0268.JPG',
 'S10_M08_R1_IMAG0269.JPG']

In [26]:
df1=df[~df.image_id.isin(nonelist)]

In [27]:
len(df1)

2582

In [28]:
train_df, validate_df = train_test_split(df1, test_size=0.20, random_state=523)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [29]:
len(train_df)

2065

In [30]:
len(validate_df)

517

In [31]:
imglist = train_df['image_id'].tolist()

In [32]:
X_train = np.empty((len(imglist),227,227,3))

In [33]:
Y_train = np.empty((len(imglist),1))

In [34]:
X_train.shape

(2065, 227, 227, 3)

In [35]:
Y_train.shape

(2065, 1)

In [36]:
path = '/root/data/'

In [37]:
imglist[:2]

['S10/P13/P13_R3/S10_P13_R3_IMAG0767.JPG',
 'S10/M09/M09_R3/S10_M09_R3_IMAG0588.JPG']

In [ ]:
count=0
nonelist=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
#         img = image.img_to_array(img)/255.0
        X_train[count]=img
#         Y_train[count]=np.array((1,0))
        count+=1
    except:
        nonelist.append(img_path)

In [70]:
len(nonelist)

0

In [71]:
X_train[:1]

array([[[[0.34117648, 0.44705883, 0.62352943],
         [0.35294119, 0.48627451, 0.63529414],
         [0.3764706 , 0.47058824, 0.65098041],
         ...,
         [0.30980393, 0.45490196, 0.65882355],
         [0.30980393, 0.41176471, 0.64313728],
         [0.34117648, 0.4509804 , 0.63921571]],

        [[0.08627451, 0.06666667, 0.09019608],
         [0.35294119, 0.4627451 , 0.65098041],
         [0.36862746, 0.45490196, 0.63921571],
         ...,
         [0.35294119, 0.44313726, 0.64705884],
         [0.32156864, 0.44313726, 0.66666669],
         [0.34509805, 0.45490196, 0.64313728]],

        [[0.31764707, 0.3137255 , 0.38431373],
         [0.34509805, 0.47843137, 0.62352943],
         [0.39607844, 0.46666667, 0.65490198],
         ...,
         [0.32549021, 0.44313726, 0.64705884],
         [0.34117648, 0.44313726, 0.64313728],
         [0.34117648, 0.44705883, 0.65490198]],

        ...,

        [[0.86274511, 0.86274511, 0.86274511],
         [0.86274511, 0.86274511, 0.86274511]

In [99]:
label = train_df['label']
Y_train = [i for i in label]

In [100]:
type(Y_train)

list

In [101]:
type(X_train)

numpy.ndarray

In [102]:
d=np.array(Y_train)

In [103]:
d.shape

(2065,)

In [104]:
Y_train=d.reshape(2065,1)

In [105]:
Y_train.shape

(2065, 1)

In [106]:
train_df[:2]

,image_id,label
0,S10/P13/P13_R3/S10_P13_R3_IMAG0861.JPG,5
1,S10/M09/M09_R3/S10_M09_R3_IMAG0570.JPG,21


In [107]:
Y_train[:2]

array([[ 5],
       [21]])

In [84]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

validate_df

In [108]:
imglist = validate_df['image_id'].tolist()
X_validate = np.empty((len(imglist),227,227,3))
X_validate.shape

(517, 227, 227, 3)

In [109]:
count=0
nonelist_validate=[]
for img_name in imglist:
    img_path = path + img_name
    try:
        img = image.load_img(img_path, target_size=(227,227))
        img = image.img_to_array(img)/255.0
        X_validate[count]=img
#         Y_train[count]=np.array((1,0))
        count+=1
    except:
        nonelist_validate.append(img_path)

In [110]:
X_validate.shape

(517, 227, 227, 3)

In [115]:
label = validate_df['label']
Y_validate = [i for i in label]
Y_validate = np.array(Y_validate)
Y_validate = Y_validate.reshape(len(Y_validate),1)
Y_validate.shape

(517, 1)

In [118]:
validate_df[:2]

,image_id,label
0,S10/P13/P13_R3/S10_P13_R3_IMAG0864.JPG,5
1,S10/M09/M09_R3/S10_M09_R3_IMAG0622.JPG,21


In [116]:
Y_validate[:2]

array([[ 5],
       [21]])

## Prepare the data 

- X_train
- Y_train
- X_validate
- Y_validate

In [126]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.datasets import cifar10
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import numpy as np


classes_num = 2
batch_size = 32
epochs_num = 100

In [130]:
def train():
    # ????
    (x_train, y_train), (x_test, y_test) = (X_train,Y_train), (X_validate, Y_validate)

    # ???????
#     y_train = keras.utils.to_categorical(y_train, classes_num)
#     y_test = keras.utils.to_categorical(y_test, classes_num)
#     # ??????
#     x_train = x_train.astype('float32')
#     x_test = x_test.astype('float32')

    datagan = ImageDataGenerator(rescale=1.)

    # 
    conv_base = VGG16(include_top=False, weights='imagenet')

    # 
    sample_count = len(y_train)
    train_features = np.zeros(shape=(sample_count, 4, 4, 512))
    train_labels = np.zeros(shape=(sample_count, classes_num))
    train_generator = datagan.flow(x_train, y_train, batch_size=batch_size)
    i = 0
    for inputs_batch, labels_batch in train_generator:
        features_batch = conv_base.predict(inputs_batch)
        train_features[i * batch_size : (i + 1) * batch_size] = features_batch
        train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # train_features = np.reshape(train_features, (sample_count, 4*4*512))
    
    # ??????????????????
    sample_count = len(y_test)
    test_generator = datagan.flow(x_test, y_test, batch_size=batch_size)
    test_features = np.zeros(shape=(sample_count, 4, 4, 512))
    test_labels = np.zeros(shape=(sample_count, classes_num))
    i = 0
    for inputs_batch, labels_batch in test_generator:
        features_batch = conv_base.predict(inputs_batch)
        test_features[i * batch_size : (i + 1) * batch_size] = features_batch
        test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # test_features = np.reshape(test_features, (sample_count, 4*4*512))
    # ??????????????????
    sample_count = len(y_test)
    test_generator = datagan.flow(x_test, y_test, batch_size=batch_size)
    test_features = np.zeros(shape=(sample_count, 4, 4, 512))
    test_labels = np.zeros(shape=(sample_count, classes_num))
    i = 0
    for inputs_batch, labels_batch in test_generator:
        features_batch = conv_base.predict(inputs_batch)
        test_features[i * batch_size : (i + 1) * batch_size] = features_batch
        test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # test_features = np.reshape(test_features, (sample_count, 4*4*512))
    
    
    model = quality_classify_model()

    # hist = model.fit_generator(train_datagan.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch = 8000, epochs = epochs_num, validation_data=(x_test,y_test), shuffle=True)
    hist = model.fit(train_features, train_labels, batch_size=batch_size, epochs=epochs_num, validation_data=(test_features, test_labels))

    model.save('./extract_features/cifar10_model.hdf5') 
    model.save_weights('./extract_features/cifar10_model_weight.hdf5')

    hist_dict = hist.history
    print("train acc:")
    print(hist_dict['acc'])
    print("validation acc:")
    print(hist_dict['val_acc'])

In [122]:
def quality_classify_model():
    model = Sequential()
    model.add(Flatten(input_shape=(4,4,512)))# 4*4*512
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_num, activation='softmax'))  # ???

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [131]:
train()

ValueError: could not broadcast input array from shape (32,7,7,512) into shape (32,4,4,512)

In [136]:
# -- coding: utf-8 --
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
from keras.datasets import cifar10
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
import numpy as np


classes_num = 2
batch_size = 32
epochs_num = 100

def quality_classify_model():
    model = Sequential()
    model.add(Flatten(input_shape=(4,4,512)))# 4*4*512
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes_num, activation='softmax'))  # ???

    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def train():
    # ????
    x_train, y_train  = X_train, Y_train
    x_test, y_test = X_validate, Y_validate

    
    # ???????
    y_train = keras.utils.to_categorical(y_train, classes_num)
    y_test = keras.utils.to_categorical(y_test, classes_num)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    datagan = ImageDataGenerator(rescale=1.)

    # ??????????
    conv_base = VGG16(include_top=False, weights='imagenet')

    # ??????????????????
    sample_count = len(y_train)
    train_features = np.zeros(shape=(sample_count, 4, 4, 512))
    train_labels = np.zeros(shape=(sample_count, classes_num))
    train_generator = datagan.flow(x_train, y_train, batch_size=batch_size)
    i = 0
    for inputs_batch, labels_batch in train_generator:
        features_batch = conv_base.predict(inputs_batch)
        train_features[i * batch_size : (i + 1) * batch_size] = features_batch
        train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # train_features = np.reshape(train_features, (sample_count, 4*4*512))

    # ??????????????????
    sample_count = len(y_test)
    test_generator = datagan.flow(x_test, y_test, batch_size=batch_size)
    test_features = np.zeros(shape=(sample_count, 4, 4, 512))
    test_labels = np.zeros(shape=(sample_count, classes_num))
    i = 0
    for inputs_batch, labels_batch in test_generator:
        features_batch = conv_base.predict(inputs_batch)
        test_features[i * batch_size : (i + 1) * batch_size] = features_batch
        test_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    # test_features = np.reshape(test_features, (sample_count, 4*4*512))

    model = quality_classify_model()

    # hist = model.fit_generator(train_datagan.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch = 8000, epochs = epochs_num, validation_data=(x_test,y_test), shuffle=True)
    hist = model.fit(train_features, train_labels, batch_size=batch_size, epochs=epochs_num, validation_data=(test_features, test_labels))

    model.save('./extract_features/cifar10_model.hdf5') 
    model.save_weights('./extract_features/cifar10_model_weight.hdf5')

    hist_dict = hist.history
    print("train acc:")
    print(hist_dict['acc'])
    print("validation acc:")
    print(hist_dict['val_acc'])

    train_acc = hist.history['acc']
    val_acc = hist.history['val_acc']
    train_loss = hist.history['loss']
    val_loss = hist.history['val_loss']

    # ??
    epochs = range(1, len(train_acc)+1)
    plt.plot(epochs, train_acc, 'bo', label = 'Training acc')
    plt.plot(epochs, val_acc, 'r', label = 'Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.savefig("accuracy.png")
    plt.figure() # ?????
    plt.plot(epochs, train_loss, 'bo', label = 'Training loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    plt.savefig("loss.png")
    

In [161]:
x_train, y_train  = X_train, Y_train
x_test, y_test = X_validate, Y_validate    

In [159]:
y_train = keras.utils.to_categorical(y_train, 22)
# y_test = keras.utils.to_categorical(y_test, classes_num)

TypeError: 'list' object cannot be interpreted as an integer

In [162]:
y_train

array([[ 5],
       [21],
       [ 5],
       ...,
       [21],
       [21],
       [ 5]])

In [156]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)